In [1]:
%load_ext autoreload
%autoreload 2

# <div class="alert-info">TCDF</div>

In [2]:
import sys

sys.path.append("/home/lorena/Documents/Lorena_Thesis/Causal_CNN/")

from libs_TCDF.runTCDF import runTCDF

In [3]:
import os, time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import torch
import pickle
import matplotlib.pyplot as plt

from libs.network_analysis import scores, evaluation

In [4]:
data_types = ['synthetic', 'curated']
results_path = './Results/Results_TCDF/'

In [5]:
for d_type in data_types:
    
    data_path = './beeline_data/' + d_type + '/'
    
    if d_type == 'synthetic':
        names = ['dyn-BF', 'dyn-BFC', 'dyn-CY', 'dyn-LI', 'dyn-LL', 'dyn-TF']
    if d_type == 'curated':
        names = ['VSC', 'mCAD', 'HSC', 'GSD']
        
    data = {}
    refs = {}

    for name in names:
    
        base_path = data_path + name + '/'
        data = {}
        refs = {}
        links = {}
    
        for folder in os.listdir(base_path):
            temp_path = os.path.join(base_path, folder)
            if os.path.isdir(temp_path):
                gem = pd.read_csv(temp_path + '/ExpressionData.csv', index_col = 0) #read gem
                pseudotime = pd.read_csv(temp_path + '/PseudoTime.csv', index_col = 0) #is distributed in columns, we need to make it one
                pseudotime_all = pd.DataFrame({'pseudotime': pseudotime.fillna(0).sum(axis=1)}) #make one pseudotime
                gem = gem.T #transpose
                gem['pseudotime'] = pseudotime_all.values #add pseudotime to gem
            
                data[folder] = gem.sort_values(by='pseudotime').drop('pseudotime', axis=1) #sort gem according to pseudotime
                refs[folder] = pd.read_csv(temp_path + '/refNetwork.csv') #ground truth
            
        #Training the model
    
        for n_levels in np.arange(3):
            times = []
            for folder in data:
                start = time.time()
                links[folder], _, _ = runTCDF(data[folder], cuda=True, levels=n_levels)
                end = time.time()
                times.append(end-start)
                torch.cuda.empty_cache()
        
            with open(results_path + 'links/' + name + '_' + str(n_levels) + '_links.pickle', 'wb') as handle:
                pickle.dump(links, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
            metrics_df = pd.DataFrame()
        
            for el in links:
                tfs = data[el].columns
                genes = tfs.copy()
                metrics_df = metrics_df.append(evaluation(links[el], refs[el], tfs, genes))
            
            metrics_df.index = links.keys()
            metrics_df["time"] = np.round(times,2)
            
            metrics_df.to_csv(results_path + name + '_' + str(n_levels) +'_levels.csv')
        


 Analysis started for target:  g1
Epoch:  1 [0%] 	Loss: 1.232730
Epoch: 500 [50%] 	Loss: 0.118044
Epoch: 1000 [100%] 	Loss: 0.083686

 Analysis started for target:  g2
Epoch:  1 [0%] 	Loss: 1.193361
Epoch: 500 [50%] 	Loss: 0.190736
Epoch: 1000 [100%] 	Loss: 0.183161

 Analysis started for target:  g3
Epoch:  1 [0%] 	Loss: 1.035882
Epoch: 500 [50%] 	Loss: 0.281080
Epoch: 1000 [100%] 	Loss: 0.268550

 Analysis started for target:  g4
Epoch:  1 [0%] 	Loss: 0.457535
Epoch: 500 [50%] 	Loss: 0.160494
Epoch: 1000 [100%] 	Loss: 0.121831

 Analysis started for target:  g6
Epoch:  1 [0%] 	Loss: 0.547519
Epoch: 500 [50%] 	Loss: 0.121092
Epoch: 1000 [100%] 	Loss: 0.115122

 Analysis started for target:  g7
Epoch:  1 [0%] 	Loss: 0.213691
Epoch: 500 [50%] 	Loss: 0.033144
Epoch: 1000 [100%] 	Loss: 0.032693

 Analysis started for target:  g8
Epoch:  1 [0%] 	Loss: 1.422642
Epoch: 500 [50%] 	Loss: 0.077435
Epoch: 1000 [100%] 	Loss: 0.074462

 Analysis started for target:  g1
Epoch:  1 [0%] 	Loss: 1.895